In [ ]:
!pip install ultralytics fiftyone opencv-python matplotlib


import cv2
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
import yaml
import fiftyone as fo
import fiftyone.zoo as foz
import random
import yaml
import importlib.resources as pkg_resources


In [ ]:

def load_dataset_cfg(name):
    """Loads an Ultralytics dataset YAML from ultralytics/cfg/datasets."""
    path = pkg_resources.files("ultralytics") / "cfg" / "datasets" / name
    path = str(path)
    assert os.path.isfile(path), f"{path} not found"
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg, path

def normalize_names(names):
    """Ensure names is a list of strings, regardless of dict/list form."""
    if isinstance(names, dict):
        return [names[i] for i in range(len(names))]
    elif isinstance(names, list):
        return names
    else:
        raise ValueError("Unsupported names format")

def resolve_split(cfg, key):
    """
    Resolve a split key ('train', 'val', 'test') to an absolute images dir.
    Handles path + relative split path.
    """
    base_path = cfg.get("path", "") or ""
    split = cfg[key]
    if isinstance(split, list):
        split = split[0]
    if base_path and not os.path.isabs(split):
        split = os.path.join(base_path, split)
    return os.path.abspath(split)

# Load the original configs
coco_cfg, coco_yaml_path = load_dataset_cfg("coco128.yaml")
home_cfg, home_yaml_path = load_dataset_cfg("HomeObjects-3K.yaml")

coco_names = normalize_names(coco_cfg["names"])
home_names = normalize_names(home_cfg["names"])

print("Num COCO classes:", len(coco_names))
print("Num HomeObjects classes:", len(home_names))


In [ ]:
# Build union names
union_names = list(coco_names)  # start with COCO classes
for hn in home_names:
    if hn not in union_names:
        union_names.append(hn)

print("Num union classes:", len(union_names))

# Map: HomeObjects local ID -> union ID
home_to_union = {}
for i, hn in enumerate(home_names):
    union_id = union_names.index(hn)
    home_to_union[i] = union_id

print("Example HomeObjects mapping:", list(home_to_union.items())[:10])

# (optional) old prints
coco_train_images_guess = resolve_split(coco_cfg, "train")
coco_val_images_guess   = resolve_split(coco_cfg, "val")
home_train_images = resolve_split(home_cfg, "train")
home_val_images   = resolve_split(home_cfg, "val")

print("COCO train images (guess):", coco_train_images_guess)
print("COCO val images (guess):  ", coco_val_images_guess)
print("Home train images:", home_train_images)
print("Home val images:  ", home_val_images)


In [ ]:
import glob

def remap_homeobjects_labels_to_union(home_cfg, home_to_union):
    """
    Find all 'labels' directories under the HomeObjects-3K dataset root and
    remap their YOLO label files to use union class IDs.
    """
    # Dataset root is whatever 'path' is in HomeObjects-3K.yaml
    home_root = home_cfg.get("path", "")
    if not home_root:
        # Fallback: assume images path's parent is root
        from pathlib import Path
        home_train_images_local = resolve_split(home_cfg, "train")
        home_root = str(Path(home_train_images_local).parents[1])

    home_root = os.path.abspath(home_root)
    print(f"[INFO] HomeObjects root (for labels search): {home_root}")

    # 1. Discover all 'labels' dirs that contain .txt files
    label_dirs = []
    for dirpath, dirnames, filenames in os.walk(home_root):
        if os.path.basename(dirpath) == "labels":
            txt_files = glob.glob(os.path.join(dirpath, "**", "*.txt"), recursive=True)
            if txt_files:
                label_dirs.append(dirpath)
                print(f"[INFO] Found labels dir: {dirpath} ({len(txt_files)} files)")

    if not label_dirs:
        print("[WARN] No labels directories with .txt files found under", home_root)
        return

    # 2. Remap class IDs
    for labels_dir in label_dirs:
        txt_files = glob.glob(os.path.join(labels_dir, "**", "*.txt"), recursive=True)
        print(f"[INFO] Remapping labels in {labels_dir} ({len(txt_files)} files)")

        for lf in txt_files:
            with open(lf, "r") as f:
                lines = f.readlines()

            new_lines = []
            changed = False

            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue

                cls_id = int(float(parts[0]))
                if cls_id in home_to_union:
                    new_cls = home_to_union[cls_id]
                    parts[0] = str(new_cls)
                    changed = True

                new_lines.append(" ".join(parts))

            if changed:
                with open(lf, "w") as f:
                    f.write("\n".join(new_lines) + "\n")

    print("[INFO] Finished remapping HomeObjects labels to union IDs")


In [ ]:
remap_homeobjects_labels_to_union(home_cfg, home_to_union)


In [ ]:
from ultralytics.data.utils import check_det_dataset

# Make sure coco128 is downloaded and get its info
data_info = check_det_dataset("coco128.yaml")  # downloads if missing
print("check_det_dataset keys:", data_info.keys())

coco_train_images = os.path.abspath(data_info["train"])
coco_val_images   = os.path.abspath(data_info["val"])

print("Resolved COCO train images:", coco_train_images)
print("Resolved COCO val images:  ", coco_val_images)

# Reconfirm HomeObjects paths (using same resolve_split as earlier)
home_train_images = resolve_split(home_cfg, "train")
home_val_images   = resolve_split(home_cfg, "val")

print("Home train images:", home_train_images)
print("Home val images:  ", home_val_images)


In [ ]:
from ultralytics.data.utils import check_det_dataset
import os, yaml

# Ensure COCO128 is downloaded and get its real paths
coco_info = check_det_dataset("coco128.yaml")  # downloads if needed
coco_train_images = os.path.abspath(coco_info["train"])
coco_val_images   = os.path.abspath(coco_info["val"])

print("Resolved COCO train images:", coco_train_images)
print("Resolved COCO val images:  ", coco_val_images)

# Ensure HomeObjects-3K is downloaded and get its real paths
home_info = check_det_dataset("HomeObjects-3K.yaml")  # downloads if needed
home_train_images = os.path.abspath(home_info["train"])
home_val_images   = os.path.abspath(home_info["val"])

print("Resolved Home train images:", home_train_images)
print("Resolved Home val images:  ", home_val_images)


In [ ]:
# union_names should already exist from earlier
union_names_dict = {i: name for i, name in enumerate(union_names)}

def write_yaml(path, data):
    with open(path, "w") as f:
        yaml.safe_dump(data, f, sort_keys=False)
    print("[INFO] Wrote", path)

# 1) Combined COCO + HomeObjects for training
coco_home_union = {
    "path": "",
    "train": [coco_train_images, home_train_images],
    "val":   [coco_val_images,   home_val_images],
    "names": union_names_dict,
}
write_yaml("coco_home_union.yaml", coco_home_union)

# 2) COCO-only (for eval), with union names
coco_union = {
    "path": "",
    "train": coco_train_images,
    "val":   coco_val_images,
    "names": union_names_dict,
}
write_yaml("coco_union.yaml", coco_union)

# 3) HomeObjects-only (for eval), with union names
homeobjects_union = {
    "path": "",
    "train": home_train_images,
    "val":   home_val_images,
    "names": union_names_dict,
}
write_yaml("homeobjects_union.yaml", homeobjects_union)


# Remap home objects

In [ ]:
import glob, os

from ultralytics.data.utils import check_det_dataset

# 1) Ask Ultralytics where HomeObjects-3K really lives
home_info = check_det_dataset("HomeObjects-3K.yaml")
home_train_images = os.path.abspath(home_info["train"])
home_val_images   = os.path.abspath(home_info["val"])

print("Home train images:", home_train_images)
print("Home val images:  ", home_val_images)

# 2) Derive labels root by replacing 'images' with 'labels'
def images_to_labels_dir(images_dir):
    if "images" in images_dir:
        return images_dir.replace(os.sep + "images", os.sep + "labels")
    else:
        return os.path.join(os.path.dirname(images_dir), "labels")

home_labels_root_train = images_to_labels_dir(home_train_images)
home_labels_root_val   = images_to_labels_dir(home_val_images)

print("Home train labels root:", home_labels_root_train)
print("Home val labels root:  ", home_labels_root_val)

# 3) Collect all label files and list unique IDs used
label_files = glob.glob(os.path.join(home_labels_root_train, "**", "*.txt"), recursive=True)
label_files += glob.glob(os.path.join(home_labels_root_val, "**", "*.txt"), recursive=True)

used_ids = set()
for lf in label_files:
    with open(lf, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cid = int(float(parts[0]))
            used_ids.add(cid)

print("Class IDs used in HomeObjects labels:", sorted(used_ids))


In [ ]:
# This assumes you still have `home_to_union` dict and `home_cfg` in memory.
# If not, reload HomeObjects-3K.yaml and rebuild them as before.

def remap_homeobjects_labels_to_union_labels(home_labels_roots, home_to_union):
    """
    Remaps class IDs in YOLO label files under the given roots using home_to_union mapping.
    """
    import glob

    for root in home_labels_roots:
        if not os.path.isdir(root):
            print(f"[WARN] Labels root not found: {root}")
            continue

        txt_files = glob.glob(os.path.join(root, "**", "*.txt"), recursive=True)
        print(f"[INFO] Remapping {len(txt_files)} label files under {root}...")

        for lf in txt_files:
            with open(lf, "r") as f:
                lines = f.readlines()

            new_lines = []
            changed = False

            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue

                old_id = int(float(parts[0]))
                # Only remap HomeObjects IDs; leave anything else alone
                if old_id in home_to_union:
                    new_id = home_to_union[old_id]
                    parts[0] = str(new_id)
                    changed = True

                new_lines.append(" ".join(parts))

            if changed:
                with open(lf, "w") as f:
                    f.write("\n".join(new_lines) + "\n")

    print("[INFO] Finished remapping HomeObjects labels to union IDs.")


# Call it with the TRAIN + VAL label roots discovered above
home_labels_roots = [home_labels_root_train, home_labels_root_val]
remap_homeobjects_labels_to_union_labels(home_labels_roots, home_to_union)


In [ ]:
#find IDs for every class

for i, name in enumerate(union_names):
    print(i, name)


In [ ]:

for i, hname in enumerate(home_names):
    uid = home_to_union[i]
    print(f"Home {i}: {hname} -> union {uid}: {union_names[uid]}")



In [ ]:
KEEP_IDS = [0, 15, 16, 24, 25, 26, 28, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]

In [ ]:
model = YOLO("yolov8n.pt")

for i, m in enumerate(model.model.model):
    print(i, m.__class__.__name__)

In [ ]:
num_modules = len(list(model.model.model))
print(num_modules)

In [ ]:

model = YOLO("yolov8n.pt")  # COCO-pretrained starting point

#calculate how many layers are in model (for freezing)
num_modules = len(list(model.model.model))

#best so far
model.train(
    data="coco_home_union.yaml",
    epochs=50,          # fewer epochs
    #classes=KEEP_IDS,
    imgsz=640,
    batch=16,
    lr0=0.001,          # lower base LR (default is ~0.01)
    lrf=0.01,           # stronger decay, ends very small
    freeze=num_modules-2,          # freeze layers
    patience=10,
    project="runs_union",
    name="yolov8n_union_coco_home_gentle",
)

In [ ]:
#gets the best.pt path incase it changed

print(glob.glob("runs_union/**/weights/best*.pt", recursive=True))


In [ ]:
from ultralytics import YOLO

# Load the union-trained model

base_model = YOLO("yolov8n.pt")
union_model = YOLO("runs_union/yolov8n_union_coco_home_gentle/weights/best.pt")



def print_metrics(metrics, label=""):
    print(f"{label} MODEL PERFORMANCE")
    print(f"mAP@50:     {metrics.box.map50:.3f}")
    print(f"mAP@50-95:  {metrics.box.map:.3f}")
    print(f"Precision:  {metrics.box.mp:.3f}")
    print(f"Recall:     {metrics.box.mr:.3f}")
    print(f"F1 Score:   {metrics.box.f1[0]:.3f}")
    print("-" * 40)




In [ ]:
'''
# COCO (base model)
metrics_coco = base_model.val(data="coco_union.yaml", verbose=False)
print_metrics(metrics_coco, "BASE MODEL on COCO128")

# HomeObjects performance (base model)
metrics_home = base_model.val(data="homeobjects_union.yaml", verbose=False)
print_metrics(metrics_home, "BASE MODEL on HomeObjects-3K")
'''


# COCO (fine-tuned model)
metrics_coco = union_model.val(data="coco_union.yaml", verbose=False)
print_metrics(metrics_coco, "UNION MODEL on COCO128")

# HomeObjects performance (fine-tuned model)
metrics_home = union_model.val(data="homeobjects_union.yaml", verbose=False)
print_metrics(metrics_home, "UNION MODEL on HomeObjects-3K")


# Save to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Example for Ultralytics YOLOv8
import shutil

# Path to the trained weights in Colab's temporary storage
source_best_weights = 'runs_union/yolov8n_union_coco_home_gentle/weights/best.pt'

# Destination path in Google Drive
destination_folder = '/content/drive/MyDrive/BestYoloModel'

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

# Copy the best weights
shutil.copy(source_best_weights, os.path.join(destination_folder, 'Yolo_best5.pt'))

print(f"YOLO model weights saved to {destination_folder}")



# Predict Video

In [ ]:

def test_yolo_video(model, video_path):
    """
    Test YOLOv8 model on a video.

    Args:
        model: The loaded YOLOv8 model.
        video_path: The path to the input video file.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference on the frame
        results = model(frame, classes=KEEP_IDS)#, conf=0.2)
        res = results[0]

        # Display the frame with detections
        plt.imshow(res.plot())
        plt.axis('off')
        plt.show()

    cap.release()
    print("Video processing finished.")

In [ ]:
#test_yolo_video(YOLO("runs_union/yolov8n_union_coco_home_gentle/weights/best.pt"), "/content/classroom.mp4")

test_yolo_video(YOLO("runs_union/yolov8n_union_coco_home_gentle/weights/best.pt"), "/content/215475_small.mp4")

# Checking images for visual if classification is working

In [ ]:
'''
metrics_home = union_model.val(
    data="homeobjects_union.yaml",
    classes=KEEP_IDS,
    save=True,
    project="runs_union",
    name="home_val_keep",
    verbose=False,
)
'''

# Evaluate ONLY 'person' on COCO
person_id = 0  # from your printed union names
metrics_person_coco = union_model.val(
    data="coco_union.yaml",
    classes=[person_id],
    save=True,
    project="runs_union",
    name="coco_person_debug",
    verbose=False,
)

print_metrics(metrics_person_coco, "UNION model on COCO (person only)")


In [ ]:
from pathlib import Path
from IPython.display import Image, display

viz_dir = Path(metrics_person_coco.save_dir)
image_paths = sorted(viz_dir.rglob("*.jpg"))
print("Found", len(image_paths), "COCO person debug images")

for p in image_paths[:10]:
    print(p)
    display(Image(filename=str(p)))


In [ ]:
print("Save directory:", getattr(metrics_home, "save_dir", "no save_dir attr"))


In [ ]:
import glob
from IPython.display import Image, display

viz_dir = "/content/runs/detect/val8"  # same as project/name above

image_paths = sorted(glob.glob(f"{viz_dir}/*.jpg"))  # or *.png if needed

print("Found", len(image_paths), "images")

# Show the first few
for img_path in image_paths[:10]:
    print(img_path)
    display(Image(filename=img_path))
